# SLME - Workflows

To calculate the SLME, we require the absorption coefficient of the material, as well as the direct and indirect band gap. As the CuAu and chalcopyrite materials we investigate all have direct band gaps in the $\Gamma$ point, we can obtain a very accurate band gap from the optics calculation itself.

In [2]:
# Notebook header

import os
from pymatgen import Structure
from custodian.vasp.handlers import VaspErrorHandler
from vscworkflows.config import load_config
from vscworkflows.fireworks.core import OptimizeFW, OpticsFW
from vscworkflows.workflows.core import _set_up_functional_params, \
    _set_up_relative_directory, Workflow

First we will define the directory in which we will perform the calculations on the cluster. Note that if you want to run this notebook to replicate my results, you will have to adjust this directory!

In [3]:
cluster = "breniac"

# Set up the scratch dir
if cluster == "breniac":
    scratch_dir = "/scratch/leuven/202/vsc20248/slme/test"   # This should be changed to your directory
else:
    scratch_dir = "/scratch/antwerpen/202/vsc20248/slme/test"   # This should be changed to your directory

Next we'll load the launchpad on which we will store the fireworks. If you haven't configured a launchpad yet, open a terminal and use `vsc config launchpad` to do so. The command below loads the 'base' launchpad (i.e. the default). Change this if necessary.

In [4]:
# Load the base launchpad
lpad = load_config("launchpad", cluster)                # Load whichever launchpad you have configured 

After running the [`structures.ipynb`](structures.ipynb) notebook, you should have all structure files stored in the `structures` directory. The line of code below makes a list of all the `.json` files in this directory.

In [5]:
structure_files = [file for file in os.listdir("structures") if ".json" in file]

To submit the workflows to the mongoDB server specified by the `Launchpad` defined above, we will loop over all of the structure files and generate a workflow using the `get_wf_optics` method. This is done in the cell below.

In [5]:
# Functional
functional=("hse06", {})
number_nodes = 8

# Set the error handlers
handlers = [
    VaspErrorHandler(errors_subset_to_catch=["zbrent"])
]

# Change default settings, for e.g. parallelization
user_incar_settings = {
    "KPAR": 2,
    "NPAR": 7
}

user_kpoints_settings_optimize = {"reciprocal_density": 100}
user_kpoints_settings_optics = {"gamma_density": 500}

for file in structure_files[10:]:
    
    structure = Structure.from_file(os.path.join("structures", file)) 
    directory = os.path.join(scratch_dir, 
                             file.split(".")[0].split("_")[1],
                             file.split(".")[0].split("_")[0])
    
    # Add number of nodes to spec, or "none"
    if number_nodes is not None and number_nodes != 0:
        spec = {"_fworker": str(number_nodes) + "nodes"}
    else:
        spec = {}
    
    # 1. Set up the geometry optimization with PBE
    vasp_input_params = _set_up_functional_params(
        functional=("pbe", {})
    )
    spec.update(
        {"_launch_dir": _set_up_relative_directory(directory, ("pbe", {}), "optimize")}
    )

    # Override the settings with the user specifications
    if user_kpoints_settings_optimize is not None:
        vasp_input_params["user_kpoints_settings"] = user_kpoints_settings_optimize
    if user_incar_settings is not None:
        vasp_input_params["user_incar_settings"].update(user_incar_settings)

    # Set up the Firework
    optimize_fw_pbe = OptimizeFW(structure=structure,
                             vasp_input_params=vasp_input_params,
                             custodian=handlers,
                             spec=spec)

    # 2. Set up the geometry optimization with chosen functional
    vasp_input_params = _set_up_functional_params(functional)
    spec.update(
        {"_launch_dir": _set_up_relative_directory(directory, functional, "optimize")}
    )

    # Override the settings with the user specifications
    if user_kpoints_settings_optimize is not None:
        vasp_input_params["user_kpoints_settings"] = user_kpoints_settings_optimize
    if user_incar_settings is not None:
        vasp_input_params["user_incar_settings"].update(user_incar_settings)

    # Set up the Firework
    optimize_fw = OptimizeFW(parents=optimize_fw_pbe,
                             vasp_input_params=vasp_input_params,
                             custodian=handlers,
                             spec=spec)

    # 3. Set up the optics calculation
    vasp_input_params = _set_up_functional_params(functional)
    spec.update({"_launch_dir": _set_up_relative_directory(directory, functional,
                                                           "optics")})
    
    # Override the settings with the user specifications
    if user_kpoints_settings_optics is not None:
        vasp_input_params["user_kpoints_settings"] = user_kpoints_settings_optics
    if user_incar_settings is not None:
        vasp_input_params["user_incar_settings"].update(user_incar_settings)

    # Set up the geometry optimization Firework
    optics_fw = OpticsFW(
        parents=optimize_fw,
        vasp_input_params=vasp_input_params,
        custodian=handlers,
        spec=spec
    )

    # Set up a clear name for the workflow
    workflow_name = "Optics "
    workflow_name += str(structure.composition.reduced_formula).replace(" ", "")
    workflow_name += " " + str(functional[0])

    # Create the workflow
    lpad.add_wf(
        Workflow(fireworks=[optimize_fw_pbe, optimize_fw, optics_fw],
                 links_dict={optimize_fw_pbe: [optimize_fw], optimize_fw: [optics_fw]},
                 name=workflow_name)
    )

2019-11-25 14:35:25,617 INFO Added a workflow. id_map: {-3: 1262, -2: 1263, -1: 1264}
2019-11-25 14:35:25,707 INFO Added a workflow. id_map: {-6: 1265, -5: 1266, -4: 1267}
2019-11-25 14:35:25,795 INFO Added a workflow. id_map: {-9: 1268, -8: 1269, -7: 1270}
2019-11-25 14:35:25,886 INFO Added a workflow. id_map: {-12: 1271, -11: 1272, -10: 1273}
2019-11-25 14:35:25,982 INFO Added a workflow. id_map: {-15: 1274, -14: 1275, -13: 1276}
2019-11-25 14:35:26,078 INFO Added a workflow. id_map: {-18: 1277, -17: 1278, -16: 1279}
2019-11-25 14:35:26,166 INFO Added a workflow. id_map: {-21: 1280, -20: 1281, -19: 1282}
2019-11-25 14:35:26,261 INFO Added a workflow. id_map: {-24: 1283, -23: 1284, -22: 1285}
2019-11-25 14:35:26,353 INFO Added a workflow. id_map: {-27: 1286, -26: 1287, -25: 1288}
2019-11-25 14:35:26,450 INFO Added a workflow. id_map: {-30: 1289, -29: 1290, -28: 1291}
2019-11-25 14:35:26,543 INFO Added a workflow. id_map: {-33: 1292, -32: 1293, -31: 1294}
2019-11-25 14:35:26,638 INFO A